<a href="https://colab.research.google.com/github/Zimo0528/Supply-Chain/blob/main/Supply_Chain_Biopharma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biopharma

### Install and import packages

In [44]:
!pip install forex_python

from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

import numpy as np
from scipy.stats import multivariate_normal

In [14]:
%reset -f
# Install and import packages
!pip install gurobipy
!pip install tabulate

import pandas as pd
import random
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate
import datetime as dt
_empty_series = pd.Series(dtype=float)

### Raw Data

In [56]:
base_yr = 2019

exrate = {
    '2018': [3.88, 4.33, 69.63, 109.91, 19.64, 1],
    '2019': [4.33, 0.92, 71.48, 109.82, 18.65, 1],
    '2020': [5.19, 0.82, 73.66, 103.24, 19.90, 1],
    '2021': [5.26, 0.88, 74.28, 115.59, 20.62, 1],
    '2022': [5.29, 0.93, 82.75, 131.12, 19.48, 1],
    '2023': [4.85, 0.91, 83.04, 140.99, 16.96, 1],
}
exrate = pd.DataFrame(exrate0 , index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18],
    'd_r': [  7, 12,  3,  8,  3, 17],
})
demand.set_index('from', inplace=True)

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

In [47]:
def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    c = CurrencyRates()
    rates = {currency: [] for currency in target_currencies}

    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    while current_date <= end_date:
        for currency in target_currencies:
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                rates[currency].append((current_date.strftime("%Y-%m"), rate))
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None))

        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return rates

# Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)

# Print the rates
for currency, rates in exchange_rates.items():
    print(f"Exchange rates for {currency} against {base_currency}:")
    for date, rate in rates:
        print(f"{date}: {rate}")
    print("\n")

Exchange rates for BRL against USD:
2019-01: 3.8812227074235808
2019-02: 3.6709964257693315
2019-03: 3.7808134938065536
2019-04: 3.8771804912780348
2019-05: 3.9267249064004286
2019-06: 3.9872657160792757
2019-07: 3.8272094457661465
2019-08: 3.8255866630424937
2019-09: 4.157212758245742
2019-10: 4.164433841071756
2019-11: 3.989316814794865
2019-12: 4.230468038608632
2020-01: 4.0196724230016025
2020-02: 4.266829533116178
2020-03: 4.485014120433634
2020-04: 5.2440563277249455
2020-05: 5.384792203015815
2020-06: 5.3324937027707815
2020-07: 5.442857142857142
2020-08: 5.167032410533423
2020-09: 5.433052473512972
2020-10: 5.6008339006126615
2020-11: 5.779363993845102
2020-12: 5.311915106951871
2021-01: 5.193953223046206
2021-02: 5.442320423700762
2021-03: 5.579025968638513
2021-04: 5.631619274646687
2021-05: 5.346548584671412
2021-06: 5.202126789366054
2021-07: 4.960871760350051
2021-08: 5.1060465898578755
2021-09: 5.152407548447152
2021-10: 5.415172413793104
2021-11: 5.650889618241493
2021-1

# Minimize cost using Gurobi Binary and Integer optimizer

## Functions to calculate cost, unmet demand, and excess capacity

In [49]:
# identify number of supply and demand location for iterations
n_ctry = range(demand.shape[0])
n_lines = range(demand.shape[1]+1)

# Objective function to calculate cost
def calc_total_cost(dec_plant, dec_h, dec_r, sample_exrate, base_yr=2019, tariff=0):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    # Adjust the cost using exchange rate of give year
    reindx = exrate.loc[:, f'{base_yr}'] / sample_exrate

    pcosts_rev = pcosts.values * reindx.values.reshape(-1,1)
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)
    # return pcosts_rev
    # pcosts_rev = adj_pcosts_exrate(2019, 2023)

    duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)
    duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

    return total_cost


# Calculate excess capacity given decision variables
def calc_excess_cap(dec_plant, dec_h, dec_r):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    excess_cap = (x_plant * caps.values).copy()
    excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
    excess_cap[:, 1] -= np.sum(x_h, axis=1)
    excess_cap[:, 2] -= np.sum(x_r, axis=1)
    return excess_cap

# Calculate unmet demand given decision variables
def calc_unmet_demand(dec_h, dec_r):
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    x_h_sum = np.sum(x_h, axis=0)
    x_r_sum = np.sum(x_r, axis=0)
    unmet_demand = (demand.values).copy()
    unmet_demand = np.column_stack((x_h_sum - unmet_demand[:, 0], x_r_sum - unmet_demand[:, 1]))

    return unmet_demand


## Gurobi optimizer

1. Sample 100 draws from the exchange rate distribution and for each draw determine the optimal network configuration

In [57]:
# Convert the data into a NumPy array for analysis
# Note: This example assumes all currencies have the same number of rates and ignores missing values

# Extract rates for each currency, ensuring they're in the same order for each month
rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# Ensure no NaN values; this method requires complete data
if np.isnan(currency_data).any():
    print("Data contains NaN values. Please handle missing data before proceeding.")
else:
    # Estimate the mean and covariance
    mean_vector = np.mean(currency_data, axis=0)
    covariance_matrix = np.cov(currency_data, rowvar=False)

    # Fit the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix)

sample_exrates = mvn_distribution.rvs(size=100, random_state = 42)
ones_column = np.ones((sample_exrates.shape[0], 1))

# Add the column of ones to the original array
sample_exrates = np.hstack((sample_exrates, ones_column))

In [58]:
all_plants = []
all_hs = []
all_rs = []

for i in range(100):
  sample_exrate = sample_exrates[i]

  # Set tariff value
  tariff = 0

  # Create a Gurobi model
  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
  dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}

  # Define decision variables
  dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
  dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")


  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, sample_exrate, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  # Extract results to print as table
  op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index)

  plants = []
  hs = []
  rs = []
  plants.extend(op_plant['Plant'].values)
  hs.extend(op_plant['H'].values)
  rs.extend(op_plant['R'].values)
  all_plants.append(plants)
  all_hs.append(hs)
  all_rs.append(rs)

In [62]:
column_names = ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.']
df_plants = pd.DataFrame(all_plants)
df_hs = pd.DataFrame(all_hs)
df_rs = pd.DataFrame(all_rs)

# Assign new column names to the DataFrame
df_plants.columns = column_names
df_hs.columns = column_names
df_rs.columns = column_names

In [63]:
open_close_summary = df_plants.mean()

# Identify correlations
correlation_matrix = df_plants.corr()

print("Open/Close Status Summary:")
print(open_close_summary)

print("\nCorrelation Matrix:")
print(correlation_matrix)

Open/Close Status Summary:
Brazil     0.99
Germany    1.00
India      1.00
Japan      0.72
Mexico     0.95
U.S.       0.94
dtype: float64

Correlation Matrix:
           Brazil  Germany  India     Japan    Mexico      U.S.
Brazil   1.000000      NaN    NaN  0.161165 -0.023057 -0.025392
Germany       NaN      NaN    NaN       NaN       NaN       NaN
India         NaN      NaN    NaN       NaN       NaN       NaN
Japan    0.161165      NaN    NaN  1.000000 -0.143066 -0.157552
Mexico  -0.023057      NaN    NaN -0.143066  1.000000 -0.057961
U.S.    -0.025392      NaN    NaN -0.157552 -0.057961  1.000000


In [64]:
open_close_summary = df_hs.mean()

# Identify correlations
correlation_matrix = df_hs.corr()

print("Open/Close Status Summary:")
print(open_close_summary)

print("\nCorrelation Matrix:")
print(correlation_matrix)

Open/Close Status Summary:
Brazil     0.99
Germany    0.23
India      1.00
Japan      0.08
Mexico     0.95
U.S.       0.82
dtype: float64

Correlation Matrix:
           Brazil   Germany  India     Japan    Mexico      U.S.
Brazil   1.000000  0.054929    NaN  0.029637 -0.023057 -0.047088
Germany  0.054929  1.000000    NaN -0.161165 -0.201705 -0.486150
India         NaN       NaN    NaN       NaN       NaN       NaN
Japan    0.029637 -0.161165    NaN  1.000000 -0.270604 -0.437505
Mexico  -0.023057 -0.201705    NaN -0.270604  1.000000  0.011943
U.S.    -0.047088 -0.486150    NaN -0.437505  0.011943  1.000000


In [65]:
open_close_summary = df_rs.mean()

# Identify correlations
correlation_matrix = df_rs.corr()

print("Open/Close Status Summary:")
print(open_close_summary)

print("\nCorrelation Matrix:")
print(correlation_matrix)

Open/Close Status Summary:
Brazil     0.99
Germany    0.79
India      1.00
Japan      0.72
Mexico     0.95
U.S.       0.23
dtype: float64

Correlation Matrix:
           Brazil   Germany  India     Japan    Mexico      U.S.
Brazil   1.000000 -0.051818    NaN  0.161165 -0.023057  0.054929
Germany -0.051818  1.000000    NaN  0.115922 -0.005632 -0.418299
India         NaN       NaN    NaN       NaN       NaN       NaN
Japan    0.161165  0.115922    NaN  1.000000 -0.143066  0.182056
Mexico  -0.023057 -0.005632    NaN -0.143066  1.000000  0.016354
U.S.     0.054929 -0.418299    NaN  0.182056  0.016354  1.000000


## With the threshold of 0.5

In [168]:
sample_exrates_test = mvn_distribution.rvs(size=100, random_state = 32)
ones_column = np.ones((sample_exrates_test.shape[0], 1))

# Add the column of ones to the original array
sample_exrates_test = np.hstack((sample_exrates_test, ones_column))

cost_list = []
# Create a Gurobi model
for k in range(100):
  sample_exrate = sample_exrates_test[k]

  # Set tariff value
  tariff = 0

  base_yr = 2019

  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  op_plant_status = [[1, 1, 1], [1, 0, 1], [1, 1, 1], [1, 0, 1], [1, 1, 1], [1, 1, 0]]
  op_plant_status = {(i, j): op_plant_status[i][j] for i in n_ctry for j in n_lines}
  dec_h = {(i, j): 1 for i in n_ctry for j in n_ctry}
  dec_r = {(i, j): 1 for i in n_ctry for j in n_ctry}

  # Define decision variables
  fixed_op_plant_vars = {}
  for key, value in op_plant_status.items():
      i, j = key  # Extract plant and line indices
      # Create a binary decision variable with lb and ub set to the op_plant status
      fixed_op_plant_vars[key] = model.addVar(vtype=GRB.BINARY, lb=value, ub=value, name=f"FixedOpPlant_{i}_{j}")

  dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  for i in n_ctry:  # Iterate over countries
          if op_plant[(i, 0)] == 0:  # If the plant in country i is closed
              for j in n_ctry:  # Apply for both lines
                  model.addConstr(dec_h[(i, j)] == 0, f"Close_dec_h_{i}_{j}")
                  model.addConstr(dec_r[(i, j)] == 0, f"Close_dec_r_{i}_{j}")
          # Else, if the plant is open, you may apply other constraints or leave as is for optimization

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(fixed_op_plant_vars, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(fixed_op_plant_vars, dec_h, dec_r, sample_exrate, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  # Extract results to print as table
  op_h = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  cost_list.append(round(model.objVal,2))
print("mean cost:", np.mean(cost_list))

mean cost: 1212.5049


## [[1, 1, 1], [1, 0, 1], [1, 1, 1], [0, 0, 0], [1, 1, 1], [1, 1, 1]]

In [169]:
sample_exrates_test = mvn_distribution.rvs(size=100, random_state = 32)
ones_column = np.ones((sample_exrates_test.shape[0], 1))

# Add the column of ones to the original array
sample_exrates_test = np.hstack((sample_exrates_test, ones_column))

cost_list = []
# Create a Gurobi model
for k in range(100):
  sample_exrate = sample_exrates_test[k]

  # Set tariff value
  tariff = 0

  base_yr = 2019

  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  op_plant_status = [[1, 1, 1], [1, 0, 1], [1, 1, 1], [0, 0, 0], [1, 1, 1], [1, 1, 1]]
  op_plant_status = {(i, j): op_plant_status[i][j] for i in n_ctry for j in n_lines}
  dec_h = {(i, j): 1 for i in n_ctry for j in n_ctry}
  dec_r = {(i, j): 1 for i in n_ctry for j in n_ctry}

  # Define decision variables
  fixed_op_plant_vars = {}
  for key, value in op_plant_status.items():
      i, j = key  # Extract plant and line indices
      # Create a binary decision variable with lb and ub set to the op_plant status
      fixed_op_plant_vars[key] = model.addVar(vtype=GRB.BINARY, lb=value, ub=value, name=f"FixedOpPlant_{i}_{j}")

  dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  for i in n_ctry:  # Iterate over countries
          if op_plant[(i, 0)] == 0:  # If the plant in country i is closed
              for j in n_ctry:  # Apply for both lines
                  model.addConstr(dec_h[(i, j)] == 0, f"Close_dec_h_{i}_{j}")
                  model.addConstr(dec_r[(i, j)] == 0, f"Close_dec_r_{i}_{j}")
          # Else, if the plant is open, you may apply other constraints or leave as is for optimization

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(fixed_op_plant_vars, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(fixed_op_plant_vars, dec_h, dec_r, sample_exrate, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  # Extract results to print as table
  op_h = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  cost_list.append(round(model.objVal,2))
print("mean cost:", np.mean(cost_list))

mean cost: 1217.6933


## [[1, 1, 1], [1, 1, 0], [1, 1, 1], [1, 0, 1], [1, 1, 1], [1, 0, 1]]

In [170]:
sample_exrates_test = mvn_distribution.rvs(size=100, random_state = 32)
ones_column = np.ones((sample_exrates_test.shape[0], 1))

# Add the column of ones to the original array
sample_exrates_test = np.hstack((sample_exrates_test, ones_column))

cost_list = []
# Create a Gurobi model
for k in range(100):
  sample_exrate = sample_exrates_test[k]

  # Set tariff value
  tariff = 0

  base_yr = 2019

  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  op_plant_status = [[1, 1, 1], [1, 1, 0], [1, 1, 1], [1, 0, 1], [1, 1, 1], [1, 0, 1]]
  op_plant_status = {(i, j): op_plant_status[i][j] for i in n_ctry for j in n_lines}
  dec_h = {(i, j): 1 for i in n_ctry for j in n_ctry}
  dec_r = {(i, j): 1 for i in n_ctry for j in n_ctry}

  # Define decision variables
  fixed_op_plant_vars = {}
  for key, value in op_plant_status.items():
      i, j = key  # Extract plant and line indices
      # Create a binary decision variable with lb and ub set to the op_plant status
      fixed_op_plant_vars[key] = model.addVar(vtype=GRB.BINARY, lb=value, ub=value, name=f"FixedOpPlant_{i}_{j}")

  dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  for i in n_ctry:  # Iterate over countries
          if op_plant[(i, 0)] == 0:  # If the plant in country i is closed
              for j in n_ctry:  # Apply for both lines
                  model.addConstr(dec_h[(i, j)] == 0, f"Close_dec_h_{i}_{j}")
                  model.addConstr(dec_r[(i, j)] == 0, f"Close_dec_r_{i}_{j}")
          # Else, if the plant is open, you may apply other constraints or leave as is for optimization

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(fixed_op_plant_vars, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(fixed_op_plant_vars, dec_h, dec_r, sample_exrate, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  # Extract results to print as table
  op_h = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  cost_list.append(round(model.objVal,2))
print("mean cost:", np.mean(cost_list))

mean cost: 1214.8643
